<a href="https://colab.research.google.com/github/david-c-brown/clinic_kpi_monthly/blob/main/rev_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from google.colab import files
from IPython.display import clear_output
from datetime import datetime

This data has to be cleaned by hand. There is a lot of inconsistency with raw output and manually entered that has to be rectified. Too many variables to pattern match, and new versions occur every month.

Once cleaned, we use a method analogous to =index(match()) to locate a specific block of text in each file

In [ ]:
file_path = "/content/drive/Shareddrives/rest/of/path/to/files"


# get all excel file names in the directory. again, different file types, we need to gather all
file_names = [f for f in os.listdir(file_path) if f.endswith('.xlsx') or f.endswith('.xls')]

# initialize an empty DataFrame to store results
result_df = pd.DataFrame()

for file in file_names:
    df = pd.read_excel(os.path.join(file_path, file), sheet_name='Master Sheet', header=None) # read the file and access the summary sheet.

    # find the row where 'PT' is located. this is our index(match()) function
    pt_index = df[df.iloc[:, 8] == 'PT'].index[0]

    # turn our above index into a dataframe to handle
    # filter rows where 'I' (9th column) has 1 to 5 letters and 'J' (10th column) is a number,
    # starting from the row after 'PT'
    df = df[(df.index > pt_index) &
            (df.iloc[:, 8].str.len().between(1, 6)) &
            (df.iloc[:, 9].apply(lambda x: pd.to_numeric(x, errors='coerce')))]


    # select only the initials and corresponding numbers in column J, and set the file name (without extension) as the index
    df = df.iloc[:, [8, 9]]
    df.index = [os.path.splitext(file)[0]] * len(df)

    # append the current DataFrame to the result DataFrame
    result_df = pd.concat([result_df, df])
    clear_output()
    print(result_df) # this lets us monitor progress, without cluttering the output

# Rename columns
result_df.columns = ['PT', 'rate_per_visit']

# Convert the 'rate_per_visit' column to a float type
result_df['rate_per_visit'] = pd.to_numeric(result_df['rate_per_visit'], errors='coerce')

# Round 'rate_per_visit' to 2 decimal places
result_df['rate_per_visit'] = result_df['rate_per_visit'].round(2)

# Clean up index by removing 'TP' and '_Master Charge Sheet 2023', leaving only the clinic name
result_df.index = result_df.index.str.replace('TP', '').str.replace('_Master Charge Sheet 2023', '')

now = datetime.now()
day = now.strftime('%d')
month = now.strftime('%B').lower()
year = now.strftime('%Y')

# save to parent folder
result_df.to_csv(f'{file_path}/rate_per_visit_{month}_{year}.csv')

#save to local machine
files.download(f'{file_path}/rate_per_visit_{month}_{year}.csv')
